<a href="https://colab.research.google.com/github/Orangefox76200/EMD-Capsule-LSTM/blob/main/EMD_Capsule_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EMD-Capsule-LSTM with Attention Mechanism**

Install pyEMD (EMD-signal 1.2.3)

In [68]:
!pip install EMD-signal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.7 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 115 kB 35.6 MB/s 


Import Packages

In [76]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from PyEMD import EMD , Visualisation
from PyEMD import EEMD
from PyEMD import CEEMDAN
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import LSTM
from keras.layers import Dense

Stock Index Dataset: DJIA (Jan 04,2010-Dec 31,2019)

Upload **Orangefox76200/EMD-Capsule-LSTM/DJIA.xlsx** (Github) to Google Drive, then import it from Google Drive to EMD_Capsule_LSTM.ipynb

In [ ]:
from google.colab import drive
drive.mount ('gdrive')
DJIA_raw = pd.read_excel ('gdrive/MyDrive/Colab Notebooks/DJIA.xlsx')
DJIA_raw

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


,Date,Open,High,Low,Close,Volume
0,"Jan 04, 2010",10430.69,10604.97,10430.69,10583.96,179780000
1,"Jan 05, 2010",10584.56,10584.56,10522.52,10572.02,188540000
2,"Jan 06, 2010",10564.72,10594.99,10546.55,10573.68,186040000
3,"Jan 07, 2010",10571.11,10612.37,10505.21,10606.86,217390000
4,"Jan 08, 2010",10606.40,10619.40,10554.33,10618.19,172710000
...,...,...,...,...,...,...
2511,"Dec 24, 2019",28572.57,28576.80,28503.21,28515.45,86150000
2512,"Dec 26, 2019",28539.46,28624.10,28535.15,28621.39,155970000
2513,"Dec 27, 2019",28675.34,28701.66,28608.98,28645.26,182280000
2514,"Dec 30, 2019",28654.76,28664.69,28428.98,28462.14,181600000


In [ ]:
DJIA_raw.shape

(2516, 6)

In [ ]:
DJIA_raw.ndim

2

In [ ]:
DJIA_raw.iloc[:,[0,1,5]]

,Date,Open,Volume
0,"Jan 04, 2010",10430.69,179780000
1,"Jan 05, 2010",10584.56,188540000
2,"Jan 06, 2010",10564.72,186040000
3,"Jan 07, 2010",10571.11,217390000
4,"Jan 08, 2010",10606.40,172710000
...,...,...,...
2511,"Dec 24, 2019",28572.57,86150000
2512,"Dec 26, 2019",28539.46,155970000
2513,"Dec 27, 2019",28675.34,182280000
2514,"Dec 30, 2019",28654.76,181600000


In [ ]:
DJIA_raw.dtypes

Date       object
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [ ]:
DJIA_raw.describe()

,Open,High,Low,Close,Volume
count,2516.000000,2516.000000,2516.000000,2516.000000,2.516000e+03
mean,17601.997731,17686.674499,17513.448593,17606.741574,1.895687e+08
std,5148.425951,5162.804510,5131.962418,5147.050108,1.106612e+08
min,9686.480000,9770.870000,9614.320000,9686.480000,8.410000e+06
25%,12977.190000,13044.985000,12921.337500,12981.207500,1.028275e+08
50%,17011.075000,17081.805000,16943.140000,17008.230000,1.516350e+08
75%,21426.617500,21488.690000,21383.437500,21411.107500,2.602025e+08
max,28675.340000,28701.660000,28608.980000,28645.260000,9.005100e+08


Use the close price as the time series to be predicted.

In [ ]:
DJIA_timeseries = DJIA_raw.Close
DJIA_timeseries.shape

(2516,)

In [ ]:
DJIA_timeseries.ndim

1

In [ ]:
DJIA_timeseries.iloc[:]

0       10583.96
1       10572.02
2       10573.68
3       10606.86
4       10618.19
          ...   
2511    28515.45
2512    28621.39
2513    28645.26
2514    28462.14
2515    28538.44
Name: Close, Length: 2516, dtype: float64

Empirical Mode Decomposition of close price

In [78]:
emd = EMD()
emd.emd(DJIA_timeseries)
imfs, res = emd.get_imfs_and_residue()
vis = Visualisation()
vis.plot_imfs(imfs=imfs, residue=res, include_residue=True)
vis.plot_instant_freq(imfs=imfs)
vis.show()

ValueError: ignored